In [1]:
import numpy as np # linear algebra
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('max_colwidth', None)

import seaborn as sns
import matplotlib.pyplot as plt



In [2]:
# train=pd.read_csv('/kaggle/input/ames-housing-dataset/AmesHousing.csv')
test =pd.read_csv('./dataset/test.csv')
train=pd.read_csv('./dataset/train.csv')
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
train.shape, test.shape

((1460, 81), (1459, 80))

In [4]:
training=pd.concat([train,test], axis=0, sort=False)

In [5]:
training = training.drop(['Id'] , axis = 1)

In [6]:
# Separating Target and Features

target = train['SalePrice']
target_log = np.log1p(target)
train = train.drop(['Id', 'SalePrice'], axis = 1)


test = test.drop(['Id'],axis = 1)

train_test = pd.concat([train,test], axis=0, sort=False)
print(train_test.shape)

(2919, 79)


In [7]:
nan=pd.DataFrame(train_test.isna().sum(),columns=['Nan_sum'])
nan['feat']=nan.index
nan=nan[nan['Nan_sum']>0]
nan['Percentage']=(nan['Nan_sum']/1460)*100
nan['Percentage of total data']=(nan['Nan_sum']/5459)*100
nan=nan.sort_values(by=['Nan_sum'])
nan.insert(0,'Serial No.',range(1,len(nan)+1))
nan.head()


,Serial No.,Nan_sum,feat,Percentage,Percentage of total data
TotalBsmtSF,1,1,TotalBsmtSF,0.068493,0.018318
GarageArea,2,1,GarageArea,0.068493,0.018318
GarageCars,3,1,GarageCars,0.068493,0.018318
KitchenQual,4,1,KitchenQual,0.068493,0.018318
Electrical,5,1,Electrical,0.068493,0.018318


In [8]:
# Converting non-numeric predictors stored as numbers into string

train_test['MSSubClass'] = train_test['MSSubClass'].apply(str)
train_test['YrSold'] = train_test['YrSold'].apply(str)
train_test['MoSold'] = train_test['MoSold'].apply(str)
train_test['OverallQual'] = train_test['OverallQual'].apply(str)
train_test['OverallCond'] = train_test['OverallCond'].apply(str)

In [9]:
# Filling Categorical NaN (That we know how to fill due to the description file )

train_test['Functional'] = train_test['Functional'].fillna('Typ')
train_test['Electrical'] = train_test['Electrical'].fillna("SBrkr")
train_test['KitchenQual'] = train_test['KitchenQual'].fillna("TA")
train_test['Exterior1st'] = train_test['Exterior1st'].fillna(train_test['Exterior1st'].mode()[0])
train_test['Exterior2nd'] = train_test['Exterior2nd'].fillna(train_test['Exterior2nd'].mode()[0])
train_test['SaleType'] = train_test['SaleType'].fillna(train_test['SaleType'].mode()[0])
train_test["PoolQC"] = train_test["PoolQC"].fillna("None")
train_test["Alley"] = train_test["Alley"].fillna("None")
train_test['FireplaceQu'] = train_test['FireplaceQu'].fillna("None")
train_test['Fence'] = train_test['Fence'].fillna("None")
train_test['MiscFeature'] = train_test['MiscFeature'].fillna("None")
for col in ('GarageArea', 'GarageCars'):
    train_test[col] = train_test[col].fillna(0)
        
for col in ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']:
    train_test[col] = train_test[col].fillna('None')
    
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    train_test[col] = train_test[col].fillna('None')
    
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtFullBath', 'BsmtHalfBath', 'MasVnrArea','BsmtUnfSF', 'TotalBsmtSF'):
    train_test[col] = train_test[col].fillna(0)

train_test['LotFrontage'] = train_test['LotFrontage'].fillna(train['LotFrontage'].median())
  

In [10]:
 # Checking the features with NaN remained out
for col in train_test:
    if train_test[col].isna().sum() > 0:
        print(col, train_test[col][1])
        print("--"*20)

MSZoning 1    RL
1    RL
Name: MSZoning, dtype: object
----------------------------------------
Utilities 1    AllPub
1    AllPub
Name: Utilities, dtype: object
----------------------------------------
MasVnrType 1       None
1    BrkFace
Name: MasVnrType, dtype: object
----------------------------------------
GarageYrBlt 1    1976.0
1    1958.0
Name: GarageYrBlt, dtype: float64
----------------------------------------


In [11]:
train_test["SqFtPerRoom"] = train_test["GrLivArea"] / (train_test["TotRmsAbvGrd"] +
                                                       train_test["FullBath"] +
                                                       train_test["HalfBath"] +
                                                       train_test["KitchenAbvGr"])

train_test['Total_Home_Quality'] = train_test['OverallQual'] + train_test['OverallCond']

train_test['Total_Bathrooms'] = (train_test['FullBath'] + (0.5 * train_test['HalfBath']) +
                               train_test['BsmtFullBath'] + (0.5 * train_test['BsmtHalfBath']))

train_test["HighQualSF"] = train_test["1stFlrSF"] + train_test["2ndFlrSF"]
train_test['renovated']=train_test['YearRemodAdd']+train_test['YearBuilt']

In [12]:
# Removing the useless variables

useless = ['GarageYrBlt','YearRemodAdd'] 
train_test = train_test.drop(useless, axis = 1)

In [13]:
# Creating dummy variables from categorical features

train_test_dummy = pd.get_dummies(train_test)
from scipy.stats import skew
numeric_features = train_test_dummy.dtypes[train_test_dummy.dtypes != object].index
skewed_features = train_test_dummy[numeric_features].apply(lambda x: skew(x)).sort_values(ascending=False)
high_skew = skewed_features[skewed_features > 0.5]
skew_index = high_skew.index

In [14]:
# Normalize skewed features using log_transformation
    
for i in skew_index:
    train_test_dummy[i] = np.log1p(train_test_dummy[i] )

In [15]:
nan=pd.DataFrame(train_test_dummy.isna().sum(),columns=['Nan_sum'])
nan['feat']=nan.index
nan=nan[nan['Nan_sum']>0]
nan['Percentage']=(nan['Nan_sum']/1460)*100
nan['Perc']=(nan['Nan_sum']/5459)*100
nan=nan.sort_values(by=['Nan_sum'])
nan.insert(0,'Serial No.',range(1,len(nan)+1))
nan

,Serial No.,Nan_sum,feat,Percentage,Perc


In [16]:
inf=pd.DataFrame(np.isinf(train_test_dummy).sum() ,columns=['Inf_sum'])
inf['feat']=inf.index
inf=inf[inf['Inf_sum']>0]
inf=inf.sort_values(by=['Inf_sum'])
inf.insert(0,'Serial No.',range(1,len(inf)+1))
inf

,Serial No.,Inf_sum,feat


In [17]:
# import shap
from xgboost import XGBRegressor
from catboost import Pool
from sklearn.svm import SVR
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeRegressor
from mlxtend.regressor import StackingRegressor
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error

In [18]:
# 10 Fold Cross validation

kf = KFold(n_splits=11, random_state=42, shuffle=True)

cv_scores = []
cv_std = []

baseline_models = ['Linear_Reg.','Bayesian_Ridge_Reg.','LGBM_Reg.','SVR',
                   'Dec_Tree_Reg.','Random_Forest_Reg.', 'XGB_Reg.',
                   'Grad_Boost_Reg.','Cat_Boost_Reg.','Stacked_Reg.','Stacked_Reg2']

In [19]:
# Train-Test separation

X_train = train_test_dummy[0:len(train)]
X_test = train_test_dummy[len(train):]


# Creation of the RMSE metric:
    
def rmse(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

def cv_rmse(model):   #,X_train, target_log, kf):
    rmse = np.sqrt(-cross_val_score(model, X_train, target_log, scoring="neg_mean_squared_error", cv=kf))
    return (rmse)

In [20]:
# Linear Regression

lreg = LinearRegression()
score_lreg = cv_rmse(lreg)
cv_scores.append(score_lreg.mean())
cv_std.append(score_lreg.std())

# Bayesian Ridge Regression

brr = BayesianRidge(compute_score=True)
score_brr = cv_rmse(brr)
cv_scores.append(score_brr.mean())
cv_std.append(score_brr.std())

# Light Gradient Boost Regressor

l_gbm = LGBMRegressor(objective='regression')
score_l_gbm = cv_rmse(l_gbm)
cv_scores.append(score_l_gbm.mean())
cv_std.append(score_l_gbm.std())

# Support Vector Regression

svr = SVR()
score_svr = cv_rmse(svr)
cv_scores.append(score_svr.mean())
cv_std.append(score_svr.std())

# Decision Tree Regressor

dtr = DecisionTreeRegressor()
score_dtr = cv_rmse(dtr)
cv_scores.append(score_dtr.mean())
cv_std.append(score_dtr.std())

# Random Forest Regressor

rfr = RandomForestRegressor()
score_rfr = cv_rmse(rfr)
cv_scores.append(score_rfr.mean())
cv_std.append(score_rfr.std())

# XGB Regressor

xgb = XGBRegressor()
score_xgb = cv_rmse(xgb)
cv_scores.append(score_xgb.mean())
cv_std.append(score_xgb.std())

# Gradient Boost Regressor

gbr = GradientBoostingRegressor()
score_gbr = cv_rmse(gbr)
cv_scores.append(score_gbr.mean())
cv_std.append(score_gbr.std())

# Cat Boost Regressor

catb = CatBoostRegressor()
score_catb = cv_rmse(catb)
cv_scores.append(score_catb.mean())
cv_std.append(score_catb.std())

# Stacked Regressor

stack_gen = StackingRegressor(regressors=(CatBoostRegressor(),
                                          BayesianRidge()),
                              meta_regressor = CatBoostRegressor(),
                              use_features_in_secondary = True)

Learning rate set to 0.041046
0:	learn: 0.3881254	total: 170ms	remaining: 2m 50s
1:	learn: 0.3777159	total: 182ms	remaining: 1m 30s
2:	learn: 0.3675769	total: 192ms	remaining: 1m 3s
3:	learn: 0.3581886	total: 201ms	remaining: 50.1s
4:	learn: 0.3495550	total: 209ms	remaining: 41.6s
5:	learn: 0.3412206	total: 218ms	remaining: 36.1s
6:	learn: 0.3330351	total: 229ms	remaining: 32.4s
7:	learn: 0.3249369	total: 240ms	remaining: 29.8s
8:	learn: 0.3171419	total: 251ms	remaining: 27.6s
9:	learn: 0.3102424	total: 259ms	remaining: 25.7s
10:	learn: 0.3031964	total: 269ms	remaining: 24.1s
11:	learn: 0.2965269	total: 277ms	remaining: 22.8s
12:	learn: 0.2900605	total: 285ms	remaining: 21.6s
13:	learn: 0.2840259	total: 292ms	remaining: 20.6s
14:	learn: 0.2781494	total: 299ms	remaining: 19.6s
15:	learn: 0.2726132	total: 306ms	remaining: 18.8s
16:	learn: 0.2674534	total: 313ms	remaining: 18.1s
17:	learn: 0.2620061	total: 320ms	remaining: 17.4s
18:	learn: 0.2566162	total: 327ms	remaining: 16.9s
19:	learn

In [21]:
# Stacked Regressor

stack_gen2 = StackingRegressor(regressors=(CatBoostRegressor(),
                                          XGBRegressor()),
                              meta_regressor = CatBoostRegressor(),
                              use_features_in_secondary = True)

score_stack_gen2 = cv_rmse(stack_gen2)
cv_scores.append(score_stack_gen2.mean())
cv_std.append(score_stack_gen2.std())

Learning rate set to 0.041046
0:	learn: 0.3881254	total: 6.32ms	remaining: 6.32s
1:	learn: 0.3777159	total: 12.6ms	remaining: 6.3s
2:	learn: 0.3675769	total: 18.6ms	remaining: 6.2s
3:	learn: 0.3581886	total: 24.6ms	remaining: 6.12s
4:	learn: 0.3495550	total: 30.3ms	remaining: 6.03s
5:	learn: 0.3412206	total: 37.2ms	remaining: 6.17s
6:	learn: 0.3330351	total: 45.3ms	remaining: 6.43s
7:	learn: 0.3249369	total: 52.1ms	remaining: 6.46s
8:	learn: 0.3171419	total: 57.2ms	remaining: 6.3s
9:	learn: 0.3102424	total: 62.4ms	remaining: 6.17s
10:	learn: 0.3031964	total: 67.7ms	remaining: 6.08s
11:	learn: 0.2965269	total: 73.6ms	remaining: 6.06s
12:	learn: 0.2900605	total: 78.5ms	remaining: 5.96s
13:	learn: 0.2840259	total: 83.6ms	remaining: 5.89s
14:	learn: 0.2781494	total: 88.9ms	remaining: 5.84s
15:	learn: 0.2726132	total: 94.1ms	remaining: 5.79s
16:	learn: 0.2674534	total: 99.5ms	remaining: 5.75s
17:	learn: 0.2620061	total: 104ms	remaining: 5.7s
18:	learn: 0.2566162	total: 109ms	remaining: 5.63

In [24]:
sub=pd.read_csv('./dataset/sample_submission.csv')

In [25]:
stack_f2=stack_gen2.fit(X_train,target_log)
test_stack = stack_gen2.predict(X_test)

test_pre = np.expm1(test_stack)
sub['SalePrice'] = test_pre

sub.to_csv("lkh_stack2.csv", index = False, header = True)


Learning rate set to 0.04175
0:	learn: 0.3881929	total: 8.19ms	remaining: 8.19s
1:	learn: 0.3779231	total: 18.1ms	remaining: 9.05s
2:	learn: 0.3686073	total: 28.4ms	remaining: 9.44s
3:	learn: 0.3591144	total: 38.2ms	remaining: 9.52s
4:	learn: 0.3503317	total: 48.5ms	remaining: 9.66s
5:	learn: 0.3416431	total: 58.9ms	remaining: 9.76s
6:	learn: 0.3334404	total: 68.5ms	remaining: 9.72s
7:	learn: 0.3251365	total: 78.5ms	remaining: 9.74s
8:	learn: 0.3178582	total: 88.7ms	remaining: 9.77s
9:	learn: 0.3103645	total: 98.6ms	remaining: 9.76s
10:	learn: 0.3032862	total: 109ms	remaining: 9.8s
11:	learn: 0.2964634	total: 119ms	remaining: 9.81s
12:	learn: 0.2897843	total: 129ms	remaining: 9.77s
13:	learn: 0.2832734	total: 138ms	remaining: 9.7s
14:	learn: 0.2772210	total: 147ms	remaining: 9.62s
15:	learn: 0.2712503	total: 156ms	remaining: 9.61s
16:	learn: 0.2655349	total: 165ms	remaining: 9.54s
17:	learn: 0.2598683	total: 173ms	remaining: 9.45s
18:	learn: 0.2548297	total: 183ms	remaining: 9.43s
19:	

In [ ]:
# Linear Regression

lreg = LinearRegression()
score_lreg = cv_rmse(lreg)
cv_scores.append(score_lreg.mean())
cv_std.append(score_lreg.std())

# Bayesian Ridge Regression

brr = BayesianRidge(compute_score=True)
score_brr = cv_rmse(brr)
cv_scores.append(score_brr.mean())
cv_std.append(score_brr.std())

# Light Gradient Boost Regressor

l_gbm = LGBMRegressor(objective='regression')
score_l_gbm = cv_rmse(l_gbm)
cv_scores.append(score_l_gbm.mean())
cv_std.append(score_l_gbm.std())

# Support Vector Regression

svr = SVR()
score_svr = cv_rmse(svr)
cv_scores.append(score_svr.mean())
cv_std.append(score_svr.std())

# Decision Tree Regressor

dtr = DecisionTreeRegressor()
score_dtr = cv_rmse(dtr)
cv_scores.append(score_dtr.mean())
cv_std.append(score_dtr.std())

# Random Forest Regressor

rfr = RandomForestRegressor()
score_rfr = cv_rmse(rfr)
cv_scores.append(score_rfr.mean())
cv_std.append(score_rfr.std())

# XGB Regressor

xgb = XGBRegressor()
score_xgb = cv_rmse(xgb)
cv_scores.append(score_xgb.mean())
cv_std.append(score_xgb.std())

# Gradient Boost Regressor

gbr = GradientBoostingRegressor()
score_gbr = cv_rmse(gbr)
cv_scores.append(score_gbr.mean())
cv_std.append(score_gbr.std())

# Cat Boost Regressor

catb = CatBoostRegressor()
score_catb = cv_rmse(catb)
cv_scores.append(score_catb.mean())
cv_std.append(score_catb.std())

# Stacked Regressor

stack_gen = StackingRegressor(regressors=(CatBoostRegressor(),
                                          BayesianRidge()),
                              meta_regressor = CatBoostRegressor(),
                              use_features_in_secondary = True)